<a href="https://colab.research.google.com/github/wdwzyyg/ElectronCounting/blob/master/Example_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import packages

In [1]:
!pip install ElectronCounting --upgrade
import CountingNN
import torch
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 KB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0+cu116
    Uninstalling torchvision-0.14.0+cu116:
      Successfully uninstalled torchvision-0.14.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.0 requires torch==1.13.0, b

# counting function using neural network

In [2]:
# load the model 
model = torch.load(os.path.dirname(CountingNN.__file__) + '/modelweights/model13_final.pt')

from CountingNN.locator import Locator

def fastrcnn_predict(model, arr, device, process_stride, **kwargs):
  """
  Implements Faster R-CNN on a single image to detect boxes for electron events,
  then use finding maximum to assign the entry positions

  Args:
      model: the loaded fast rcnn model
      arr: array of a single image, shape [H,W]
      device: torch.device('cpu') or torch.device('cuda')
      process_stride: divide the image into pieces when applying the fast rcnn, recommend between 32 and 64.
      meanADU: optional float for mean intensity per electron (ADU), if none, will use default 241 for 200kV.
      p_list: optional list of five multiplier for model tune, if none, will use default numbers: [6, 6, 1.3, 1.5, 23]
  """
  x = arr[None, ...]
  # device =  torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  counting = Locator(model, device, process_stride, 'max', 30, None, 'dynamic_window', meanADU = kwargs.get('meanADU'), p_list=kwargs.get('p_list'))
  filtered, event_sizes =  counting.predict_sequence(x)
  filtered = filtered[0]

  return filtered


# counting function using Connected component analysis

In [ ]:
import numpy as np
from scipy.ndimage import maximum_position
from scipy.ndimage import label

def counting_filter_max(arr, threshold=20, structure = np.ones((3,3))):
  """
  Implements CCA on a single image to detect blobs,
  then use finding maximum to assign the entry positions

  Args:
      arr: array of a single image, shape [H,W]
      threshold: dark noise thresholding
  """
  image_binary = arr > threshold 
  all_labels, num = label(image_binary, structure = np.ones((3,3)))  
  m=np.ones(shape=all_labels.shape)
  obj = maximum_position(arr, all_labels, range(1,num))
  obj = np.rint(obj).astype(int)
  x = np.zeros(shape=np.shape(arr))
  x[obj[:,0],obj[:,1]]=1

  return x

# Parallel processing

By using dask, you can create parallel tasks with multiple CPU cores or GPU cores(Dask-Cuda). Just map those counting functions for a lazy signal.